# Basic Usage

In [1]:
import os

os.environ["OPENAI_API_KEY"] = "df91094d076546c08bbcd0eca4ada6fb"
os.environ['OPENAI_API_VERSION'] = '2023-12-01-preview'
os.environ['AZURE_DEPLOYMENT'] = 'gpt-35-turbo-20240510'
os.environ['AZURE_ENDPOINT'] = 'https://openai-20240510.openai.azure.com/'

## Embedding Model

In [2]:
from raptor.EmbeddingModels import SBertEmbeddingModel

embedding_model = SBertEmbeddingModel(
    "./models/multi-qa-mpnet-base-cos-v1"
)

e:\yi\NUSTM\project\FinQA4\raptor\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-07 18:44:03,722 - Loading faiss with AVX2 support.
2024-08-07 18:44:03,737 - Successfully loaded faiss with AVX2 support.
2024-08-07 18:44:04,132 - Load pretrained SentenceTransformer: ./models/multi-qa-mpnet-base-cos-v1
2024-08-07 18:44:04,560 - Use pytorch device: cpu


## Summarization Model

In [3]:
from raptor.SummarizationModels import AzureGPTSummarizationModel

summarization_model = AzureGPTSummarizationModel(
    'gpt-3.5-turbo'
)

## QA Model

In [4]:
from raptor.QAModels import AzureGPTQAModel, UnifiedQAModel

# qa_model = AzureGPTQAModel(
#     'gpt-3.5-turbo'
# )

qa_model = UnifiedQAModel(
    R"./models/unifiedqa-v2-t5-3b-1363200"
)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [15]:
qa_model.answer_question(
    '',
    'which is best conductor? \\n (a) iron (b) feather'
)

e:\yi\NUSTM\project\FinQA4\raptor\.venv\Lib\site-packages\transformers\generation\utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'iron'

## RA

In [5]:
from raptor import RetrievalAugmentation, RetrievalAugmentationConfig

ra_config = RetrievalAugmentationConfig(
    embedding_model=embedding_model,
    summarization_model=summarization_model,
    qa_model=qa_model,
)
# Initialize with default configuration. For advanced configurations, check the documentation. [WIP]
RA = RetrievalAugmentation(
    ra_config
)

2024-08-07 18:44:34,902 - Successfully initialized TreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
            Selection Mode: top_k
            Summarization Length: 100
            Summarization Model: <raptor.SummarizationModels.AzureGPTSummarizationModel object at 0x000001D1C4808110>
            Embedding Models: {'EMB': <raptor.EmbeddingModels.SBertEmbeddingModel object at 0x000001D1CFB8CD50>}
            Cluster Embedding Model: EMB
        
        Reduction Dimension: 10
        Clustering Algorithm: RAPTOR_Clustering
        Clustering Parameters: {}
        
2024-08-07 18:44:34,903 - Successfully initialized ClusterTreeBuilder with Config 
        TreeBuilderConfig:
            Tokenizer: <Encoding 'cl100k_base'>
            Max Tokens: 100
            Num Layers: 5
            Threshold: 0.5
            Top K: 5
           

In [24]:
with open('./demo/sample.txt', 'r') as file:
    text = file.read()
RA.add_documents(text)

2024-08-07 20:45:39,291 - Creating Leaf Nodes
Batches: 100%|██████████| 1/1 [00:00<00:00, 10.25it/s]
2024-08-07 20:45:45,423 - Created 35 Leaf Embeddings
2024-08-07 20:45:45,424 - Building All Nodes
2024-08-07 20:45:59,762 - Using Cluster TreeBuilder
2024-08-07 20:45:59,762 - Constructing Layer 0
2024-08-07 20:46:04,667 - Summarization Length: 100
2024-08-07 20:46:07,352 - HTTP Request: POST https://openai-20240510.openai.azure.com//openai/deployments/gpt-35-turbo-20240510/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
2024-08-07 20:46:31,190 - Node Texts Length: 765, Summarized Text Length: 100
Batches: 100%|██████████| 1/1 [00:00<00:00,  5.34it/s]
2024-08-07 20:46:34,059 - HTTP Request: POST https://openai-20240510.openai.azure.com//openai/deployments/gpt-35-turbo-20240510/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
2024-08-07 20:46:38,978 - Node Texts Length: 407, Summarized Text Length: 100
Batches: 100%|██████████| 1/1 [00:00<00:00,  5.33it

TypeError: expected string or buffer

In [22]:
RA.tree.root_nodes

{35: <raptor.tree_structures.Node at 0x1d260b7e6d0>,
 36: <raptor.tree_structures.Node at 0x1d25c42c2d0>,
 37: <raptor.tree_structures.Node at 0x1d2613ca8d0>,
 38: <raptor.tree_structures.Node at 0x1d261337410>,
 39: <raptor.tree_structures.Node at 0x1d261346450>}

In [ ]:
# Wrong
node_texts = 'back-door into the garden, and called, you tame pigeons, you turtle-doves, and all you birds beneath the sky, come and help me to pick      the good into the pot,      the bad into the crop Then two white pigeons came in by the kitchen window, and afterwards the turtle-doves, and at last all the birds beneath the sky, came whirring and crowding in, and alighted amongst the ashes\n\nAnd the pigeons nodded with their heads and began pick, pick, pick, pick, and the rest began also pick, pick, pick, pick, and gathered all the good grains into the dish   Hardly had one hour passed before they had finished, and all flew out again   Then the girl took the dish to her step-mother, and was glad, and believed that now she would be allowed to go with them to the festival\n\ndishes of lentils amongst the ashes, the maiden went through the back-door into the garden and cried, you tame pigeons, you turtle-doves, and all you birds beneath the sky, come and help me to pick      the good into the pot,      the bad into the crop Then two white pigeons came in by the kitchen-window, and afterwards the turtle-doves, and at length all the birds beneath the\n\nsky, came whirring and crowding in, and alighted amongst the ashes   And the doves nodded with their heads and began pick, pick, pick, pick, and the others began also pick, pick, pick, pick, and gathered all the good seeds into the dishes, and before half an hour was over they had already finished, and all flew out again Then the maiden was delighted, and believed that she might now go with them to the wedding\n\nthe left, and the younger at the right, and then the pigeons pecked out the other eye from each   And thus, for their wickedness and falsehood, they were punished with blindness all their days\n\n'

In [ ]:
summarized_text = "Cinderella goes to sit under a hazel tree three times a day to weep and pray. A little white bird always appears on the tree when Cinderella expresses a wish, and throws down to her what she had wished for. The king gave orders for a festival where all the beautiful young girls in the country were invited so that his son could choose a bride. Cinderella's step-sisters ask her to comb their hair, brush their shoes, and fasten their buckles for the"
node_texts = " Thrice a day cinderella went and sat beneath it, and wept and prayed, and a little white bird always came on the tree, and if cinderella expressed a wish, the bird threw down to her what she had wished for It happened, however, that the king gave orders for a festival which was to last three days, and to which all the beautiful young girls in the country were invited, in order that his son might choose himself a bride\n\n  When the two step-sisters heard that they too were to appear among the number, they were delighted, called cinderella and said, comb our hair for us, brush our shoes and fasten our buckles, for we are going to the wedding at the king's palace Cinderella obeyed, but wept, because she too would have liked to go with them to the dance, and begged her step-mother to allow her to do so\n\nthe step-sisters had gone once more, cinderella went to the hazel-tree and said -      shiver and quiver, my little tree,      silver and gold throw down over me Then the bird threw down a much more beautiful dress than on the preceding day  And when cinderella appeared at the wedding in this dress, every one was astonished at her beauty   The king's son had waited until she came, and instantly took her by the hand\n\nAnd now the bird threw down to her a dress which was more splendid and magnificent than any she had yet had, and the slippers were golden   And when she went to the festival in the dress, no one knew how to speak for astonishment   The king's son danced with her only, and if any one invited her to dance, he said this is my partner When evening came, cinderella wished to leave, and the king's\n\nhowever, took cinderella on his horse and rode away with her   As they passed by the hazel-tree, the two white doves cried -      turn and peep, turn and peep,      no blood is in the shoe,      the shoe is not too small for her,      the true bride rides with you, and when they had cried that, the two came flying down and placed themselves on cinderella's shoulders, one on the right,\n\nthe other on the left, and remained sitting there When the wedding with the king's son was to be celebrated, the two false sisters came and wanted to get into favor with cinderella and share her good fortune   When the betrothed couple went to church, the elder was at the right side and the younger at the left, and the pigeons pecked out one eye from each of them   Afterwards as they came back the elder was at\n\n"

In [ ]:
question = "How did Cinderella reach her happy ending?"
answer = RA.answer_question(question=question)
print("Answer: ", answer)

2024-08-06 20:03:28,754 - Using collapsed_tree
Batches: 100%|██████████| 1/1 [00:00<00:00, 23.81it/s]
2024-08-06 20:03:31,288 - HTTP Request: POST https://openai-20240510.openai.azure.com//openai/deployments/gpt-35-turbo-20240510/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"


Answer:  Cinderella reached her happy ending by attending the king's festival with the help of a little white bird who granted her wishes, wearing a beautiful dress and slippers thrown down to her by the bird, and capturing the heart of the prince who danced with her all night. The prince searched for her and found her, and they were married with the help of two white doves who announced that she was the true bride.


# Experiments
Reproduce paper's experiments.

## NarrativeQA

In [6]:
import os

file_path = R"F:\datasets\reading_comprehension\narrativeqa\data\test-00000-of-00008.parquet"

In [7]:
import pandas as pd

data = pd.read_parquet(file_path)
data.head(10)

,document,question,answers
0,{'id': '0025577043f5090cd603c6aea60f26e2361955...,"{'text': 'Who is Mark Hunter?', 'tokens': ['Wh...",[{'text': 'He is a high school student in Phoe...
1,{'id': '0025577043f5090cd603c6aea60f26e2361955...,{'text': 'Where does this radio station take p...,[{'text': 'It takes place in Mark's parents ba...
2,{'id': '0025577043f5090cd603c6aea60f26e2361955...,{'text': 'Why do more students tune into Mark'...,[{'text': 'Mark talks about what goes on at sc...
3,{'id': '0025577043f5090cd603c6aea60f26e2361955...,"{'text': 'Who commits suicide?', 'tokens': ['W...","[{'text': 'Malcolm.', 'tokens': ['Malcolm', '...."
4,{'id': '0025577043f5090cd603c6aea60f26e2361955...,{'text': 'What does Paige jam into her microwa...,[{'text': 'She jams her medals and accolades. ...
5,{'id': '0025577043f5090cd603c6aea60f26e2361955...,{'text': 'What does Mark do with his radio sta...,[{'text': 'He dismantles it and attaches it to...
6,{'id': '0025577043f5090cd603c6aea60f26e2361955...,{'text': 'What does Mark tell the protesting s...,[{'text': 'He tells them to make their own fut...
7,{'id': '0025577043f5090cd603c6aea60f26e2361955...,"{'text': 'Who gets arrested?', 'tokens': ['Who...","[{'text': 'Mark and Nora.', 'tokens': ['Mark',..."
8,{'id': '0025577043f5090cd603c6aea60f26e2361955...,"{'text': 'What does the radio show cause?', 't...","[{'text': 'It causes trouble. ', 'tokens': ['I..."
9,{'id': '0025577043f5090cd603c6aea60f26e2361955...,{'text': 'Where does Mark Broadcast his statio...,"[{'text': 'Parent's Basement', 'tokens': ['Par..."


In [8]:
print(data.loc[0, 'document'].keys())

dict_keys(['id', 'kind', 'url', 'file_size', 'word_count', 'start', 'end', 'summary', 'text'])


In [9]:
RA.add_documents(data.loc[0, 'document']['text'])

2024-08-07 17:23:13,547 - Creating Leaf Nodes
Batches: 100%|██████████| 1/1 [00:00<00:00,  9.05it/s]
2024-08-07 17:23:42,992 - Created 137 Leaf Embeddings
2024-08-07 17:23:42,992 - Building All Nodes
2024-08-07 17:23:50,408 - Using Cluster TreeBuilder
2024-08-07 17:23:53,042 - Constructing Layer 0
2024-08-07 17:26:18,107 - Summarization Length: 100
2024-08-07 17:27:06,997 - HTTP Request: POST https://openai-20240510.openai.azure.com//openai/deployments/gpt-35-turbo-20240510/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
2024-08-07 17:27:48,437 - Node Texts Length: 291, Summarized Text Length: 100
Batches: 100%|██████████| 1/1 [00:00<00:00,  5.43it/s]
2024-08-07 17:28:46,793 - HTTP Request: POST https://openai-20240510.openai.azure.com//openai/deployments/gpt-35-turbo-20240510/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
2024-08-07 17:28:46,796 - Node Texts Length: 582, Summarized Text Length: 100
Batches: 100%|██████████| 1/1 [00:00<00:00,  5.30i

Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': True, 'severity': 'medium'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}


TypeError: expected string or buffer

In [ ]:
answer = RA.answer_question(
    question=data.loc[0, 'question']['text']
)
answer

# Download

In [1]:
import os

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
# https://huggingface.co/sentence-transformers/multi-qa-mpnet-base-cos-v1
!huggingface-cli download --resume-download sentence-transformers/multi-qa-mpnet-base-cos-v1 --local-dir ./models/multi-qa-mpnet-base-cos-v1

# https://huggingface.co/allenai/unifiedqa-v2-t5-3b-1363200
!huggingface-cli download --resume-download allenai/unifiedqa-v2-t5-3b-1363200 --local-dir ./models/unifiedqa-v2-t5-3b-1363200


E:\yi\NUSTM\project\FinQA4\raptor\models\unifiedqa-v2-t5-3b-1363200



Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]e:\yi\NUSTM\project\FinQA4\raptor\.venv\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Download complete. Moving file to models\unifiedqa-v2-t5-3b-1363200\README.md
Download complete. Moving file to models\unifiedqa-v2-t5-3b-1363200\tokenizer_config.json
Download complete. Moving file to models\unifiedqa-v2-t5-3b-1363200\special_tokens_map.json
Download complete. Moving file to models\unifiedqa-v2-t5-3b-1363200\generation_config.json
Download complete. Moving file to models\unifiedqa-v2-t5-3b-1363200\.gitattributes

Fetching 8 files:  12%|█▎        | 1/8 [00:00<00:06,  1.13it/s]Download complete. Moving file to models\unifiedqa-v2-t5-3b-1363200\config.json

Fetching 8 files:  38%|███▊      | 3/8 [00:01<00:01,  3.23it/s]

In [1]:
# Download datasets
import os

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
# narrative-qa: https://huggingface.co/datasets/deepmind/narrativeqa
!huggingface-cli download --repo-type dataset --resume-download deepmind/narrativeqa --local-dir F:/datasets/reading_comprehension/narrativeqa


F:\datasets\reading_comprehension\narrativeqa


e:\yi\NUSTM\project\FinQA4\raptor\.venv\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(

Fetching 39 files:   0%|          | 0/39 [00:00<?, ?it/s]Downloading 'data/narrativeqa-master.zip' to 'F:\datasets\reading_comprehension\narrativeqa\.cache\huggingface\download\data\narrativeqa-master.zip.d9fc92d5f53409f845ba44780e6689676d879c739589861b4805064513d1476b.incomplete'
Download complete. Moving file to F:\datasets\reading_comprehension\narrativeqa\data\narrativeqa-master.zip

Fetching 39 files:   8%|▊         | 3/39 [00:03<00:38,  1.07s/it]Downloading 'data/train-00008-of-00024.parquet' to 'F:\datasets\reading_comprehension\narrativeqa\.cache\huggingface\download\data\train-00008-of-00024.parquet.207b66118655892930c495da7b7a036c03d167b57b248a7c27bdfd937e1be29c.incomplete'
Down